<a href="https://colab.research.google.com/github/priihcastro/assistente-leitura-gemini/blob/master/assistente_leitura_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# Célula 1: Instalar e Configurar
# --- Parte 1: Preparar o Laboratório e o Gemini ---
print("--- Instalando ferramentas necessárias... ---")
# Instalamos as ferramentas do Google Gemini e a ferramenta mágica Gradio
!pip install -q google-generativeai gradio
print("--- Ferramentas instaladas! ---")

print("--- Configurando o Gemini... ---")
import getpass # Ferramenta para pegar sua chave secreta de forma segura
import google.generativeai as genai # Ferramenta para falar com o Gemini
import gradio as gr # Nossa ferramenta mágica para fazer a pagininha

# Passo de pegar sua chave secreta do Gemini (só aparece a primeira vez que você rodar)
# IMPORTANTE: Sua chave é secreta! Não compartilhe o link público do Colab com a chave exposta,
# ou rode essa célula apenas em um ambiente privado.
sua_chave_gemini = getpass.getpass('Digite sua chave API do Gemini e aperte Enter: ')

# Configurar o Gemini com a sua chave
genai.configure(api_key=sua_chave_gemini)

# Usar o nome do modelo que VOCÊ encontrou que funciona para texto.
# O modelo 'gemini-1.0-flash' é rápido, mas tem limites de processamento.
nome_do_modelo_texto = 'gemini-2.0-flash'
modelo_texto = None # Inicializamos como None
try:
    modelo_texto = genai.GenerativeModel(nome_do_modelo_texto)
    # Pequeno teste rápido para garantir que o modelo carregou e responde
    modelo_texto.generate_content("Teste rápido de configuração", request_options={'timeout': 5})
    print(f"Gemini configurado com o modelo: {nome_do_modelo_texto}!")
except Exception as e:
    print(f"ERRO: Não foi possível configurar ou testar o modelo Gemini '{nome_do_modelo_texto}'!")
    print(f"Verifique o nome do modelo (use genai.list_models()) e sua chave API.")
    print(f"Detalhes do erro: {e}")
    # Se der erro grave na configuração, modelo_texto continua None, e a interface não será lançada.

print("---------------------------------------------------------\n")

# Rode esta célula primeiro! Se der erro, verifique a chave e o nome do modelo.

--- Instalando ferramentas necessárias... ---
--- Ferramentas instaladas! ---
--- Configurando o Gemini... ---
Digite sua chave API do Gemini e aperte Enter: ··········
Gemini configurado com o modelo: gemini-2.0-flash!
---------------------------------------------------------



In [31]:
# Célula 2: A Função que Faz o Trabalho
# --- Parte 2: A Caixinha Mágica que Faz o Trabalho (Nossa Função Principal) ---

# Definimos o limite MÁXIMO de caracteres para evitar timeouts com o modelo Flash.
# Este limite (2000 caracteres) foi definido empiricamente para aumentar a chance de sucesso
# em todas as 3 tarefas (Resumo, Ações, Info Chave) dentro do timeout de 60 segundos.
LIMITE_MAXIMO_CARACTERES = 2000

# O comando secreto para "finalizar" a sessão na interface (não para o código Python)
COMANDO_FINALIZAR_SESSAO = "FINALIZAR_SESSAO" # Mudei o nome para ser mais claro que é a sessão na interface

# Importamos a ferramenta de expressões regulares (usada na função)
import re

def processa_texto_com_gemini(texto_para_analisar):
    """
    Esta função é chamada pelo Gradio. Ela recebe o texto, executa os 3 agentes do Gemini
    (se o texto for válido e os passos anteriores tiverem sucesso) e retorna os resultados formatados.
    Inclui validação de entrada, tratamento de erro aprimorado, parada antecipada de agentes,
    contagem de itens e status visual.
    """
    # Verifica se o modelo Gemini foi carregado corretamente no início do script
    if modelo_texto is None:
        return "Erro Interno: O modelo Gemini não foi configurado.", "", ""

    # --- Checar o comando para finalizar a sessão na interface ---
    # Se o usuário digitar o comando secreto, retornamos a mensagem de finalização
    if texto_para_analisar.strip().upper() == COMANDO_FINALIZAR_SESSAO:
        print(f"\n--- Comando de finalização de sessão '{COMANDO_FINALIZAR_SESSAO}' recebido. ---")
        return "## ✅ Sessão Finalizada. ✅\n\nCole um novo texto para começar de novo.", "", ""


    print(f"\n--- Robozinhos começando a trabalhar com texto de {len(texto_para_analisar)} caracteres! ---")

    # --- Checar se o texto de entrada é vazio ---
    if not texto_para_analisar or len(texto_para_analisar.strip()) == 0:
        print("!! Nenhum texto inserido.")
        return "## ℹ️ Por favor, cole um texto na caixa acima e clique em Enviar! 😊", "", ""

    # --- Checar o tamanho do texto ANTES de chamar o Gemini ---
    # Compara o tamanho do texto com o nosso limite definido
    if len(texto_para_analisar) > LIMITE_MAXIMO_CARACTERES:
        print(f"!! Texto muito longo! Tamanho: {len(texto_para_analisar)} caracteres. Limite: {LIMITE_MAXIMO_CARACTERES} caracteres.")
        mensagem_aviso = (
            f"## 🚨 Texto muito longo! 🚨\n\n"
            f"O texto que você colou tem **{len(texto_para_analisar)} caracteres**. "
            f"Para garantir que o Gemini Flash responda rapidinho sem timeouts, o limite atual é de aproximadamente **{LIMITE_MAXIMO_CARACTERES} caracteres**.\n\n"
            f"Por favor, reduza o tamanho do texto e tente novamente."
        )
        # Retorna a mensagem de aviso e limpa os outros campos na interface
        return mensagem_aviso, "", ""

    # --- Se o texto é válido (não vazio e dentro do limite), chamamos os Agentes do Gemini ---

    # --- Função auxiliar para contar itens de lista usando Markdown markers (- or *) ---
    def contar_itens_lista(texto):
        if not texto or not isinstance(texto, str):
            return 0
        # Usa expressão regular para encontrar linhas que começam com '-' ou '*' (ignorando espaços antes)
        itens_encontrados = re.findall(r'^\s*[-*]', texto, re.MULTILINE)
        return len(itens_encontrados)


    # --- Agente 1: Resumidor ---
    print("--- Chamando Agente Resumidor... ---")
    # Prompt para pedir o resumo, com instrução para usar negrito em pontos importantes
    pergunta_resumo = "Resuma o texto a seguir de forma concisa e clara, em português. Use negrito para os pontos mais importantes:\n" + texto_para_analisar
    resposta_resumo = "Falha ao obter resumo. 😔"
    resumo_obtidocom_sucesso = False
    status_processamento = "🔴 Falha Geral!" # Status inicial antes de chamar o 1º agente

    try:
        # Chama o modelo Gemini para gerar o resumo
        resp = modelo_texto.generate_content(pergunta_resumo, request_options={'timeout': 60}) # Timeout de 60 segundos
        if resp.text:
            resposta_resumo = resp.text.strip() # Remove espaços extras do começo/fim
            resumo_obtidocom_sucesso = True
            status_processamento = "🟡 Processando..." # Status se o resumo funcionou, continuando para os próximos agentes

        else:
             # Caso o Gemini retorne uma resposta sem texto (pode acontecer por erro interno não levantado como exceção)
             resposta_resumo = "Hummm, o Agente Resumidor não conseguiu fazer o resumo para este texto. A resposta da IA não continha texto. Tente simplificar ou reduzir mais."
             status_processamento = "🟠 Resumo sem texto."

    except Exception as e:
        # Captura erros durante a chamada ao Gemini
        if "timed out" in str(e) or "Timeout" in str(e) or "deadline exceeded" in str(e): # Verifica diferentes tipos de timeout
             resposta_resumo = (
                 "⏳ O Agente Resumidor demorou demais! (Timeout após 60 segundos).\n"
                 "Isso pode acontecer com textos que, mesmo dentro do limite, são complexos ou com instabilidade na conexão.\n"
                 "Por favor, tente reduzir **AINDA MAIS** o tamanho do texto ou simplificar as frases e tente de novo. Reiniciar o Colab pode ajudar (menu Ambiente de execução -> Reiniciar)." # Sugere reiniciar
             )
             status_processamento = "🔴 Timeout Resumo!"
        else: # Para outros tipos de erro inesperado
            resposta_resumo = f"!! Ops! Erro inesperado no Resumidor: {e}"
            status_processamento = "🔴 Erro Resumo!"
        print(f"Log: {resposta_resumo}") # Loga o erro no console do Colab


    # --- Checa se o Resumidor teve sucesso ANTES de chamar os próximos agentes ---
    # Se o resumo não foi obtido com sucesso (deu erro, timeout ou veio sem texto), paramos por aqui.
    if not resumo_obtidocom_sucesso:
        print("--- Resumo falhou ou não teve texto. Parando processamento dos outros agentes. ---")
        # Retorna a mensagem de erro/status do Resumidor para todas as caixas na interface
        # Isso deixa claro pro usuário que os outros passos não foram tentados.
        msg_parada = f"Processamento parado: {status_processamento}"
        # Formata a resposta do resumo com o status e a mensagem de erro
        return f"Status: **{status_processamento}**\n\n{resposta_resumo}", msg_parada, msg_parada

    # --- SE CHEGOU ATÉ AQUI, significa que o Resumidor funcionou! Continuamos para os outros Agentes ---
    # Atualiza o status agora que o resumo deu certo
    status_processamento = "🟢 Resumo OK!"
    resposta_acoes = "Falha ao obter ações. 😔"
    resposta_info = "Falha ao obter informações chave. 😔"
    acoes_obtidas_sucesso = False
    info_obtidas_sucesso = False


    # --- Agente 2: Extrator de Ações ---
    print("--- Chamando Agente Extrator de Ações... ---")
    # Prompt para extrair ações, pedindo formato de lista com quem/prazo, e sendo conciso
    pergunta_acoes = "Com base no texto a seguir, liste APENAS as ações ou tarefas que precisam ser realizadas. Liste usando marcadores de lista Markdown (`- `) e seja conciso. Para cada ação, diga QUEM deve fazer (se mencionado) e o PRAZO (se mencionado). Em português:\n" + texto_para_analisar

    try:
        resp = modelo_texto.generate_content(pergunta_acoes, request_options={'timeout': 60}) # Timeout de 60 segundos
        if resp.text:
             resposta_acoes = resp.text.strip()
             acoes_obtidas_sucesso = True
        else:
             resposta_acoes = "Hummm, o Agente Extrator de Ações não conseguiu encontrar tarefas para este texto. A resposta da IA não continha texto. Tente simplificar ou reduzir mais."

    except Exception as e:
       # Captura erros durante a chamada ao Gemini
       if "timed out" in str(e) or "Timeout" in str(e) or "deadline exceeded" in str(e):
            resposta_acoes = (
                 "⏳ O Agente Extrator de Ações demorou demais! (Timeout após 60 segundos).\n"
                 "Isso pode acontecer com textos que, mesmo dentro do limite, são complexos ou com instabilidade na conexão.\n"
                 "Por favor, tente reduzir **AINDA MAIS** o tamanho do texto ou simplificar as frases e tente de novo. Reiniciar o Colab pode ajudar (menu Ambiente de execução -> Reiniciar)."
             )
       else:
           resposta_acoes = f"!! Ops! Erro inesperado no Extrator de Ações: {e}"
       print(f"Log: {resposta_acoes}")

    # Conta os itens encontrados (linhas com marcadores) na resposta de Ações
    contagem_acoes = contar_itens_lista(resposta_acoes)


    # --- Agente 3: Extrator de Informações Chave ---
    print("--- Chamando Agente Extrator de Informações Chave... ---")
    # Prompt para extrair informações chave, com exemplo do formato desejado (- **Tipo:** Valor)
    pergunta_info = """
Com base no texto a seguir, liste APENAS as principais informações chave que aparecem.
Liste usando marcadores de lista Markdown (`- `).
Para cada item, inclua: Nomes de Pessoas, Nomes de Lugares/Organizações, Datas/Prazos específicos, Nomes de Projetos/Módulos.
Use o seguinte FORMATO para cada item da lista:

- **Tipo:** Valor

Exemplo:
- **Pessoa:** João Silva
- **Data:** 15/08/2024
- **Projeto:** Sistema Beta
- **Lugar:** Sala de Reunião C

Em português:
""" + texto_para_analisar

    try:
        resp = modelo_texto.generate_content(pergunta_info, request_options={'timeout': 60}) # Timeout de 60 segundos
        if resp.text:
            resposta_info = resp.text.strip()
            info_obtidas_sucesso = True
        else:
            resposta_info = "Hummm, o Agente Extrator de Informações Chave não conseguiu encontrar informações para este texto. A resposta da IA não continha texto. Tente simplificar ou reduzir mais."

    except Exception as e:
       # Captura erros durante a chamada ao Gemini
       if "timed out" in str(e) or "Timeout" in str(e) or "deadline exceeded" in str(e):
            resposta_info = (
                 "⏳ O Agente Extrator de Informações Chave demorou demais! (Timeout após 60 segundos).\n"
                 "Isso pode acontecer com textos que, mesmo dentro do limite, são complexos ou com instabilidade na conexão.\n"
                 "Por favor, tente reduzir **AINDA MAIS** o tamanho do texto ou simplificar as frases e tente de novo. Reiniciar o Colab pode ajudar (menu Ambiente de execução -> Reiniciar)."
             )
       else:
           resposta_info = f"!! Ops! Erro inesperado no Extrator de Informações Chave: {e}"
       print(f"Log: {resposta_info}")

    # Conta os itens encontrados (linhas com marcadores) na resposta de Informações Chave
    contagem_info = contar_itens_lista(resposta_info)

    print("--- Robozinhos terminaram o trabalho! ---")

    # --- Formata as saídas finais para a interface do Gradio ---

    # Atualiza o status final do processamento geral
    if acoes_obtidas_sucesso and info_obtidas_sucesso:
        status_processamento = "✅ Concluído com Sucesso!"
    elif resumo_obtidocom_sucesso: # Se o resumo deu certo, mas ações OU info falharam
         status_processamento = "⚠️ Concluído com Alerta (alguns agentes falharam)."
    # Se o resumo NÃO deu certo, o status_processamento já foi definido como erro antes.


    # Prepara a resposta do Resumo com Status no topo
    resposta_resumo_final = f"Status: **{status_processamento}**\n\n{resposta_resumo}"

    # Prepara a resposta de Ações com contagem e ícone/status
    if acoes_obtidas_sucesso:
        resposta_acoes_final = f"## ✅ **{contagem_acoes} Ação(ões) Encontrada(s):**\n\n{resposta_acoes}" # Título H2 e negrito
    else:
        # Se falhou, a resposta_acoes já contém a mensagem de erro específica
        resposta_acoes_final = f"## ❌ **Ações:**\n\n{resposta_acoes}"


    # Prepara a resposta de Informações Chave com contagem e ícone/status
    if info_obtidas_sucesso:
         resposta_info_final = f"## 📍 **{contagem_info} Informação(ões) Chave Encontrada(s):**\n\n{resposta_info}" # Título H2 e negrito
    else:
        # Se falhou, a resposta_info já contém a mensagem de erro específica
        resposta_info_final = f"## ❌ **Informações Chave:**\n\n{resposta_info}"


    # Retorna os 3 resultados formatados para as caixas de saída do Gradio
    return resposta_resumo_final, resposta_acoes_final, resposta_info_final

# Rode esta célula para ter a versão final da função processa_texto_com_gemini!

In [32]:
# Célula 3: Criar e Mostrar a Interface Mágica
# --- Parte 3 & 4: Criar e Mostrar a Pagininha Mágica com Gradio ---
print("--- Criando e Lançando a interface mágica com Gradio... ---")

# --- INSTRUÇÕES IMPORTANTES PARA O USUÁRIO DO COLAB ---
print("\n--- INSTRUÇÕES ---")
print(f"1. A interface do Assistente vai aparecer logo abaixo ou em um link público.")
print(f"2. Cole seu texto na caixa de entrada (limite aprox. {LIMITE_MAXIMO_CARACTERES} caracteres) e clique em 'Submit'.")
print(f"3. Os resultados (Resumo, Ações, Informações Chave) aparecerão nas caixas abaixo.")
print(f"4. Para LIMPAR A INTERFACE E ENCERRAR A SESSÃO ATUAL: Digite '{COMANDO_FINALIZAR_SESSAO}' na caixa de texto e clique em 'Submit'.")
print(f"5. Para PARAR A EXECUÇÃO REAL DO CÓDIGO PYTHON NO COLAB (desligar o servidor Gradio): Clique no botão quadrado [■] do lado desta célula.")
print(f"6. Se a interface não aparecer em 1 minuto após rodar esta célula, verifique os logs (saída de texto abaixo) por mensagens de erro.")
print("------------------\n")


# Lança a interface do Gradio APENAS se o modelo Gemini foi configurado sem erro na Célula 1
if modelo_texto is not None:
    interface_magica = gr.Interface(
        fn=processa_texto_com_gemini, # A função Python que o Gradio deve chamar
        # Configuração da caixa de entrada (Textbox): altura, label, etc.
        inputs=gr.Textbox(lines=15, label=f"👇 Cole seu texto aqui (limite aprox. {LIMITE_MAXIMO_CARACTERES} caracteres) e clique em Submit!"),
        # Configuração das caixas de saída (Markdown para formatar o texto)
        outputs=[
            gr.Markdown(label="✨ Resumo Rápido:"), # Caixa para o resumo formatado
            gr.Markdown(label="✅ Ações/Tarefas Encontradas:"), # Caixa para as ações formatadas
            gr.Markdown(label="📍 Informações Chave (Pessoas, Datas, Lugares...):") # Caixa para as info chave formatadas
        ],
        # Título e descrição da interface do Gradio
        title="📄✨ Assistente de Análise de Texto com Gemini Flash ✨📄", # Título que aparece na pagininha (Pode ajustar!)
        description=f"Cole um texto abaixo (limite aprox. **{LIMITE_MAXIMO_CARACTERES} caracteres**) e deixe o Gemini Flash trabalhar! Ele vai resumir, achar tarefas e extrair informações importantes para você rapidinho! 👇",
        theme="soft" # Tema visual para a interface
    )

    # Lança a interface!
    # share=True cria um link público temporário (útil para mostrar, mas cuidado se a chave API for exposta)
    # debug=True mostra logs e erros na saída da célula do Colab (MUITO útil para depurar!)
    interface_magica.launch(share=True, debug=True)

    print("\n--- Interface lançada! Procure a pagininha (ou o link público) acima ou abaixo. ---")
    # Este print só aparecerá se você parar a célula manualmente (com o botão quadrado).
    print("--- Célula de lançamento da interface finalizada. ---")

else:
    print("\n--- Interface NÃO lançada porque o modelo Gemini não foi configurado corretamente na Célula 1. ---")


print("--- Fim do Código ---") # Este print só aparecerá se você parar a célula manualmente.

# Rode esta célula para ver a interface e as instruções!

--- Criando e Lançando a interface mágica com Gradio... ---

--- INSTRUÇÕES ---
1. A interface do Assistente vai aparecer logo abaixo ou em um link público.
2. Cole seu texto na caixa de entrada (limite aprox. 2000 caracteres) e clique em 'Submit'.
3. Os resultados (Resumo, Ações, Informações Chave) aparecerão nas caixas abaixo.
4. Para LIMPAR A INTERFACE E ENCERRAR A SESSÃO ATUAL: Digite 'FINALIZAR_SESSAO' na caixa de texto e clique em 'Submit'.
5. Para PARAR A EXECUÇÃO REAL DO CÓDIGO PYTHON NO COLAB (desligar o servidor Gradio): Clique no botão quadrado [■] do lado desta célula.
6. Se a interface não aparecer em 1 minuto após rodar esta célula, verifique os logs (saída de texto abaixo) por mensagens de erro.
------------------

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://47580ecb7f8adfb72a.gradio.live

This share link expires in 1 week. For free permanent hosti


--- Robozinhos começando a trabalhar com texto de 2003 caracteres! ---
!! Texto muito longo! Tamanho: 2003 caracteres. Limite: 2000 caracteres.

--- Robozinhos começando a trabalhar com texto de 1943 caracteres! ---
--- Chamando Agente Resumidor... ---
--- Chamando Agente Extrator de Ações... ---
--- Chamando Agente Extrator de Informações Chave... ---
--- Robozinhos terminaram o trabalho! ---
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://33cf5110ecbe86819b.gradio.live
Killing tunnel 127.0.0.1:7860 <> https://47580ecb7f8adfb72a.gradio.live

--- Interface lançada! Procure a pagininha (ou o link público) acima ou abaixo. ---
--- Célula de lançamento da interface finalizada. ---
--- Fim do Código ---
